### Agentic AI PDF RAG Pipeline (Groq + FAISS + HuggingFace)
This notebook demonstrates an end-to-end RAG (Retrieval-Augmented Generation) pipeline over a 288-page Agentic AI PDF book. It includes:

📄 PDF loading and semantic chunking using LangChain's PyPDFLoader and RecursiveCharacterTextSplitter

🔍 Embedding text with all-MiniLM-L6-v2 via HuggingFaceEmbeddings

🧠 Local vector storage using FAISS with flat indexing

🔄 Retrieval of top-k relevant chunks using vector similarity

💬 Response generation using Groq's ultra-fast gemma-2b-it LLM

🧱 Prompt orchestration using LangChain's hub + RunnablePassthrough + StrOutputParser

Query Example:
"Explain the router in Agentic AI" → returns a semantically accurate answer grounded in the book content.

This pipeline is designed for fast, local prototyping of retrieval-based LLM applications with modular LangChain components.



In [1]:
# 🌍 Environment Setup
import os
from dotenv import load_dotenv
load_dotenv()
os.environ['GROQ_API_KEY'] = os.getenv('GROQ_API_KEY')

# 📄 Document Loading & Chunking
from langchain.document_loaders import PyPDFLoader
from langchain.text_splitter import RecursiveCharacterTextSplitter

# 🔍 Embedding & Vector Store
from langchain_huggingface import HuggingFaceEmbeddings
from langchain.vectorstores import FAISS

# 🤖 LLM, Prompting, and Chain Building
from langchain_groq import ChatGroq
from langchain_core.output_parsers import StrOutputParser
from langchain_core.runnables import RunnablePassthrough
from langchain import hub

In [2]:
loader = PyPDFLoader(r"data\agentic-ai-system.pdf")
pages = loader.load()
pages[111]

Document(metadata={'producer': 'Adobe PDF Library 17.0; modified using iText® 5.5.13 ©2000-2018 iText Group NV (AGPL-version)', 'creator': 'Adobe InDesign 18.5 (Windows)(Foxit Advanced PDF Editor)', 'creationdate': '2025-04-01T12:11:31+05:30', 'icnappname': 'Foxit Advanced PDF Editor', 'icnappplatform': 'Windows', 'icnappversion': '3.05', 'moddate': 'D:20250424033741', 'trapped': '/False', 'source': 'data\\agentic-ai-system.pdf', 'total_pages': 288, 'page': 111, 'page_label': '87'}, page_content='Implementing reflective capabilities 87\nWe will then set up a crew with the defined agents and tasks and kick off the process:\n1 from crewai import Agent, Task, Crew\n2 crew = Crew(\n3    agents=[preference_agent, meta_agent],\n4    tasks=[generate_recommendation, adjust_weights],\n5    verbose=True)\n6\n7 crew.kickoff()\nThe output would look something like this:\n# Agent: Travel destination recommender\n## Task: Use the recommend_destination tool with these preferences: \n{\'budget\': 0.04

In [3]:
len(pages)

288

In [4]:
pages[0]

Document(metadata={'producer': 'Adobe PDF Library 17.0; modified using iText® 5.5.13 ©2000-2018 iText Group NV (AGPL-version)', 'creator': 'Adobe InDesign 18.5 (Windows)(Foxit Advanced PDF Editor)', 'creationdate': '2025-04-01T12:11:31+05:30', 'icnappname': 'Foxit Advanced PDF Editor', 'icnappplatform': 'Windows', 'icnappversion': '3.05', 'moddate': 'D:20250424033741', 'trapped': '/False', 'source': 'data\\agentic-ai-system.pdf', 'total_pages': 288, 'page': 0, 'page_label': 'i'}, page_content='')

In [5]:
# Skip the first 12 pages (index 0–11), keep from page 13 onwards
filtered_pages = pages[12:]

In [6]:
pages[1].metadata

{'producer': 'Adobe PDF Library 17.0; modified using iText® 5.5.13 ©2000-2018 iText Group NV (AGPL-version)',
 'creator': 'Adobe InDesign 18.5 (Windows)(Foxit Advanced PDF Editor)',
 'creationdate': '2025-04-01T12:11:31+05:30',
 'icnappname': 'Foxit Advanced PDF Editor',
 'icnappplatform': 'Windows',
 'icnappversion': '3.05',
 'moddate': 'D:20250424033741',
 'trapped': '/False',
 'source': 'data\\agentic-ai-system.pdf',
 'total_pages': 288,
 'page': 1,
 'page_label': 'ii'}

In [7]:
print(pages[1].page_content)

Building Agentic AI Systems
Create intelligent, autonomous AI agents that can reason, plan, 
and adapt
Anjanava Biswas
Wrick Talukdar


In [8]:
filtered_pages[0]

Document(metadata={'producer': 'Adobe PDF Library 17.0; modified using iText® 5.5.13 ©2000-2018 iText Group NV (AGPL-version)', 'creator': 'Adobe InDesign 18.5 (Windows)(Foxit Advanced PDF Editor)', 'creationdate': '2025-04-01T12:11:31+05:30', 'icnappname': 'Foxit Advanced PDF Editor', 'icnappplatform': 'Windows', 'icnappversion': '3.05', 'moddate': 'D:20250424033741', 'trapped': '/False', 'source': 'data\\agentic-ai-system.pdf', 'total_pages': 288, 'page': 12, 'page_label': 'xiii'}, page_content='')

In [9]:
text_splitter = RecursiveCharacterTextSplitter(chunk_size=800, chunk_overlap=100, separators=['\n\n', '\n', ' ', ''])

semantic_chunks  = text_splitter.split_documents(filtered_pages)

semantic_chunks[0]

Document(metadata={'producer': 'Adobe PDF Library 17.0; modified using iText® 5.5.13 ©2000-2018 iText Group NV (AGPL-version)', 'creator': 'Adobe InDesign 18.5 (Windows)(Foxit Advanced PDF Editor)', 'creationdate': '2025-04-01T12:11:31+05:30', 'icnappname': 'Foxit Advanced PDF Editor', 'icnappplatform': 'Windows', 'icnappversion': '3.05', 'moddate': 'D:20250424033741', 'trapped': '/False', 'source': 'data\\agentic-ai-system.pdf', 'total_pages': 288, 'page': 13, 'page_label': 'xiv'}, page_content='Preface xix\nPart 1: Foundations of Generative AI and  \nAgentic Systems\n1\nFundamentals of Generative AI 3\nIntroduction to generative AI 3\nTypes of generative AI models 4\nV AEs 5\nGANs 6\nAutoregressive models and Transformer \narchitecture 7\nLLM-powered AI agents 10\nApplications of generative AI 16\nChallenges and limitations of \ngenerative AI 20\nQuality of data and bias 21\nData privacy 21\nComputational resources 22\nEthical and social implications 23\nGeneralization and creativity

In [10]:
len(semantic_chunks)

945

In [11]:
for i, chunk in enumerate(semantic_chunks[:2]):
    print(f"Chunk {i+1}:\n")
    print(chunk.page_content[:300])
    print("-" * 50)


Chunk 1:

Preface xix
Part 1: Foundations of Generative AI and  
Agentic Systems
1
Fundamentals of Generative AI 3
Introduction to generative AI 3
Types of generative AI models 4
V AEs 5
GANs 6
Autoregressive models and Transformer 
architecture 7
LLM-powered AI agents 10
Applications of generative AI 16
Chal
--------------------------------------------------
Chunk 2:

Agency 29
Autonomy 30
Example of agency and autonomy in agents 32
Reviewing intelligent agents and 
their characteristics 35
Table of Contents
--------------------------------------------------


In [12]:
embedding_hf = HuggingFaceEmbeddings(model='all-MiniLM-L6-v2')

e:\Learning\Agentic_AI_V2\1-Pydantic\venv\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [13]:
text = "Agentic AI systems require a router to delegate tasks."
vector = embedding_hf.embed_query(text)
print(vector[:5])  # Show first 5 dimensions

[-0.01841013692319393, -0.05309692770242691, -0.05101615563035011, 0.006131480913609266, 0.02873179316520691]


In [14]:
# Extract the text content from Document objects
texts = [doc.page_content for doc in semantic_chunks]

# Pass only the raw text to the embed_documents method
data_vector = embedding_hf.embed_documents(texts)


In [15]:
len(data_vector[0])

384

In [16]:
# milvus_client = MilvusClient('milvus_db.db')

# from pymilvus import MilvusClient

# client = MilvusClient("milvus_db.db")  # This now uses embedded Milvus via milvus_lite

# collection_name = "agentic_chunks"

# client.create_collection(
#     collection_name=collection_name,
#     dimension=384,
#     metric_type="COSINE"
# )



⚠️ Note: The MilvusClient("milvus_db.db") call requires milvus_lite, which is not publicly available via pip. Although pymilvus and milvus are installed, the embedded Milvus (milvus_lite) module is not included.

In [17]:
# FAISS Flat index creation (default behavior is Flat)
faiss_index = FAISS.from_documents(documents=semantic_chunks, embedding=embedding_hf)
faiss_index.save_local(r'db\faiss_agentic_index_flat')

In [18]:
faiss_index = FAISS.load_local(r'db\faiss_agentic_index_flat', embeddings=embedding_hf, allow_dangerous_deserialization=True)

In [19]:
retriever = faiss_index.as_retriever(kwargs={"k":10})

In [20]:
retriever

VectorStoreRetriever(tags=['FAISS', 'HuggingFaceEmbeddings'], vectorstore=<langchain_community.vectorstores.faiss.FAISS object at 0x00000205D51C0550>, search_kwargs={})

In [21]:
retriever.invoke("ai?")

[Document(id='b4c34659-b245-4f7e-ba72-1bf67e5bdbe7', metadata={'producer': 'Adobe PDF Library 17.0; modified using iText® 5.5.13 ©2000-2018 iText Group NV (AGPL-version)', 'creator': 'Adobe InDesign 18.5 (Windows)(Foxit Advanced PDF Editor)', 'creationdate': '2025-04-01T12:11:31+05:30', 'icnappname': 'Foxit Advanced PDF Editor', 'icnappplatform': 'Windows', 'icnappversion': '3.05', 'moddate': 'D:20250424033741', 'trapped': '/False', 'source': 'data\\agentic-ai-system.pdf', 'total_pages': 288, 'page': 268, 'page_label': '244'}, page_content='Overcoming these hurdles requires breakthroughs in areas such as causal reasoning, self-supervised \nlearning, and embodied AI – moving beyond pattern recognition to develop truly adaptive, self-\nimproving systems. While current AI models excel at narrow tasks, true AGI remains a distant goal, \nrequiring a fundamental shift in how machines learn, reason, and interact with the world.\nWhen we combine AGI with autonomous agents, we create something 

In [22]:
model = ChatGroq(model='gemma2-9b-it')

In [23]:
query = "Explain the router in Agentic AI."
results = faiss_index.similarity_search(query, k=3)
print(results[0].page_content)


• Reviewing intelligent agents and their characteristics
• Exploring the architecture of agentic systems
• Understanding multi-agent systems
By the end of this chapter, you will have an overview of the basics of intelligent agents and the most 
critical aspects of agentic system architecture that must be considered while building an intelligent 
agentic system.


In [24]:
prompt = hub.pull('rlm/rag-prompt')

def format_docs(docs):
    return "\n\n".join(doc.page_content for doc in docs)

rag_chain = (
    {"context": retriever | format_docs, 'question': RunnablePassthrough()}
    | prompt
    | model
    | StrOutputParser()
)

In [25]:
result = rag_chain.invoke("what is ai?")

### Flat + Huggingface + FAISS + Groq

In [26]:
import os
from dotenv import load_dotenv
load_dotenv()

import warnings
warnings.filterwarnings('ignore')

# --- LangChain + Integrations ---
from langchain.document_loaders import PyPDFLoader
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain_huggingface import HuggingFaceEmbeddings
from langchain.vectorstores import FAISS
from langchain_groq import ChatGroq
from langchain import hub
from langchain_core.output_parsers import StrOutputParser
from langchain_core.runnables import RunnablePassthrough

# --- Set API Key ---
os.environ['GROQ_API_KEY'] = os.getenv('GROQ_API_KEY')

# --- Load PDF & Skip Front Matter ---
loader = PyPDFLoader("data/agentic-ai-system.pdf")
pages = loader.load()[12:]  # Skip first 12 pages

# --- Semantic Chunking ---
text_splitter = RecursiveCharacterTextSplitter(
    chunk_size=800,
    chunk_overlap=100,
    separators=["\n\n", "\n", " ", ""]
)
semantic_chunks = text_splitter.split_documents(pages)

# --- Embedding & FAISS Index ---
embedding_hf = HuggingFaceEmbeddings(model="all-MiniLM-L6-v2")
faiss_index = FAISS.from_documents(semantic_chunks, embedding=embedding_hf)
faiss_index.save_local("db/faiss_agentic_index_flat")

# --- Load Index & Set Up Retriever ---
faiss_index = FAISS.load_local("db/faiss_agentic_index_flat", embeddings=embedding_hf, allow_dangerous_deserialization=True)
retriever = faiss_index.as_retriever(search_kwargs={"k": 10})

# --- Load Prompt Template & Model ---
prompt = hub.pull("rlm/rag-prompt")
model = ChatGroq(model="gemma2-9b-it")

# --- Define RAG Chain ---
def format_docs(docs):
    return "\n\n".join(doc.page_content for doc in docs)

rag_chain = (
    {"context": retriever | format_docs, "question": RunnablePassthrough()}
    | prompt
    | model
    | StrOutputParser()
)

# --- Query Example ---
result = rag_chain.invoke("What is AI?")
print(result)


AI, or artificial intelligence, refers to computer systems designed to perform tasks that typically require human intelligence, such as learning, problem-solving, and decision-making.  These systems learn from data and can adapt to new information, enabling them to perform complex tasks and improve over time. 
AI is rapidly advancing and is being applied in various fields, including healthcare, finance, and entertainment. 





### HNSW + Huggingface + FAISS + Groq

In [27]:
import os
from dotenv import load_dotenv
load_dotenv()

import warnings
warnings.filterwarnings('ignore')

from langchain.document_loaders import PyPDFLoader
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain.embeddings import HuggingFaceEmbeddings
from langchain_groq import ChatGroq
from sklearn.metrics.pairwise import cosine_similarity
from langchain import hub
from langchain_core.runnables import RunnablePassthrough


In [28]:
loader = PyPDFLoader("data/agentic-ai-system.pdf")
pages = loader.load()

pages = pages[12:]

In [29]:
text_splitter = RecursiveCharacterTextSplitter(chunk_size=1000, chunk_overlap=100)

text_splited = text_splitter.split_documents(pages)

In [30]:
text_splited[1].page_content

'Table of Contents xiv\nExploring the architecture  \nof agentic systems 36\nDeliberative architectures 36\nReactive architectures 38\nHybrid architectures 39\nUnderstanding multi-agent systems 40\nDefinition and characteristics of MASs 42\nInteraction mechanisms in MASs 43\nSummary 48\nQuestions 49\nAnswers 49\n3\nEssential Components of Intelligent Agents 51\nTechnical requirements 51\nKnowledge representation in \nintelligent agents 52\nSemantic networks 52\nFrames 54\nLogic-based representations 55\nReasoning in intelligent agents 56\nDeductive reasoning 57\nInductive reasoning 58\nAbductive reasoning 59\nLearning mechanisms for  \nadaptive agents 61\nDecision-making and planning in \nagentic systems 62\nUtility function 63\nPlanning algorithms 65\nEnhancing agent capabilities with \ngenerative AI 68\nStart building agentic AI 69\nSummary 72\nQuestions 72\nAnswers 72\nPart 2: Designing and Implementing Generative \nAI-Based Agents\n4\nReflection and Introspection  \nin Agents 77\nT

In [31]:
text_splited[110].metadata

{'producer': 'Adobe PDF Library 17.0; modified using iText® 5.5.13 ©2000-2018 iText Group NV (AGPL-version)',
 'creator': 'Adobe InDesign 18.5 (Windows)(Foxit Advanced PDF Editor)',
 'creationdate': '2025-04-01T12:11:31+05:30',
 'icnappname': 'Foxit Advanced PDF Editor',
 'icnappplatform': 'Windows',
 'icnappversion': '3.05',
 'moddate': 'D:20250424033741',
 'trapped': '/False',
 'source': 'data/agentic-ai-system.pdf',
 'total_pages': 288,
 'page': 57,
 'page_label': '33'}

In [32]:
embedding_hf = HuggingFaceEmbeddings(model_name="all-MiniLM-L6-v2")

In [33]:
sample_data = ['my name is bhagwat', 'i live in pune', 'i am a ai solution architect']

vectors = embedding_hf.embed_documents(sample_data)
vectors[0]

[-0.05215435102581978,
 -0.028457490727305412,
 -0.06018109247088432,
 0.0734371691942215,
 -0.10013759136199951,
 -0.03263907507061958,
 0.14939884841442108,
 0.006166362669318914,
 0.0018060653237625957,
 -0.021175390109419823,
 -0.0205252505838871,
 -0.14886103570461273,
 0.06308189034461975,
 -0.01124359481036663,
 0.02466544695198536,
 0.09470432251691818,
 0.012575981207191944,
 -0.054297663271427155,
 -0.08138665556907654,
 -0.08164478838443756,
 -0.04732225462794304,
 0.11897630244493484,
 -0.011571512557566166,
 -0.015285871922969818,
 -0.05452505499124527,
 -0.0017737828893586993,
 0.014821115881204605,
 0.035811059176921844,
 -0.024308891966938972,
 0.02245725318789482,
 -0.03316447138786316,
 -0.014292985200881958,
 0.02366502210497856,
 0.03701024129986763,
 -0.008929131552577019,
 0.03026132471859455,
 -0.11209756880998611,
 0.05432495102286339,
 0.013018644414842129,
 0.00465072738006711,
 -0.05469384789466858,
 -0.05322590470314026,
 -0.0038592780474573374,
 -0.03174032

In [34]:
len(vectors)

3

In [35]:
len(vectors[0])

384

In [36]:
query = 'What is the person name?'

embed_query = embedding_hf.embed_query(query)
embed_query

[-0.12938393652439117,
 0.08283582329750061,
 -0.08256246894598007,
 -0.004697174299508333,
 -0.08572892099618912,
 0.011185230687260628,
 0.15421408414840698,
 0.01532006449997425,
 0.029017232358455658,
 0.012395162135362625,
 0.01656029187142849,
 -0.09195587784051895,
 -0.019430451095104218,
 -0.02143518626689911,
 -0.08794497698545456,
 0.034275103360414505,
 -0.036462604999542236,
 0.07902806997299194,
 0.03665642440319061,
 -0.056194473057985306,
 -0.0677170529961586,
 -0.03890253230929375,
 -0.03010411188006401,
 0.0007049685809761286,
 0.019226975739002228,
 -0.0175187848508358,
 -0.030687952414155006,
 -0.03002936951816082,
 -0.010599694214761257,
 -0.03619292378425598,
 0.020409926772117615,
 -0.040945012122392654,
 0.06686566770076752,
 0.11534159630537033,
 -0.03739703819155693,
 -0.007018722593784332,
 -0.05022764205932617,
 0.11874319612979889,
 -0.017581168562173843,
 -0.043544184416532516,
 0.03803153336048126,
 -0.04004983976483345,
 -0.04223741218447685,
 -0.07027076

In [37]:
len(embed_query)

384

In [38]:
cosine_similarity([embed_query], vectors)

array([[0.32219183, 0.11770844, 0.10101418]])

In [39]:
faiss_index = FAISS.from_documents(documents=text_splited, embedding=embedding_hf)

In [40]:
faiss_index.save_local('db/faiss_agentic_index_hnsw')

In [41]:
faiss_index = faiss_index.load_local('db/faiss_agentic_index_hnsw', embedding_hf, allow_dangerous_deserialization=True)

In [42]:
type(faiss_index)

langchain_community.vectorstores.faiss.FAISS

In [43]:
retriever_hnsw = faiss_index.as_retriever(kwargs={"k":3})

In [44]:
retriever_hnsw

VectorStoreRetriever(tags=['FAISS', 'HuggingFaceEmbeddings'], vectorstore=<langchain_community.vectorstores.faiss.FAISS object at 0x00000205D66CEC30>, search_kwargs={})

In [45]:
llm = ChatGroq(model="gemma2-9b-it")
llm

ChatGroq(client=<groq.resources.chat.completions.Completions object at 0x0000020592321950>, async_client=<groq.resources.chat.completions.AsyncCompletions object at 0x0000020592323750>, model_name='gemma2-9b-it', model_kwargs={}, groq_api_key=SecretStr('**********'))

In [46]:
prompt = hub.pull("rlm/rag-prompt")

In [47]:
def format_docs(docs):
    return "\n\n".join(doc.page_content for doc in docs)

In [48]:
rag_chain = ({'context': retriever | format_docs, 'question': RunnablePassthrough()}
            | prompt
            | llm
            | StrOutputParser())

result = rag_chain.invoke('what is agentic ai?')
print(result)

Agentic AI refers to robust, scalable, and effective AI systems capable of handling complex tasks. These systems exhibit intentionality, pursuing specific goals and adjusting their actions accordingly.  Agentic AI systems are also responsible for the consequences of their decisions. 

